In [1]:
import os
import requests
from langchain.tools import Tool
from langgraph.graph import StateGraph, END

# Set your API key
API_KEY = "f668e3202e384bd086093351251003"
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

# Function to fetch weather data
def get_weather(city):
    url = f"{BASE_URL}?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather = data["weather"][0]["description"]
        temp = data["main"]["temp"]
        return f"The weather in {city} is {weather} with a temperature of {temp}°C."
    else:
        return "Unable to fetch weather data."

# Define a tool for LangGraph
weather_tool = Tool(
    name="GetWeather",
    func=get_weather,
    description="Fetches weather information for a given city."
)

# LangGraph Setup
class WeatherState:
    city: str
    response: str

graph = StateGraph(WeatherState)

@graph.node
def fetch_weather(state):
    state.response = get_weather(state.city)
    return state

graph.add_edge(fetch_weather, END)
graph.set_entry_point(fetch_weather)

weather_graph = graph.compile()

# Run the graph
state = WeatherState(city="New York")
result = weather_graph.invoke(state)
print(result.response)


AttributeError: 'StateGraph' object has no attribute 'node'

In [4]:
import requests
from langgraph.graph import StateGraph, END
from typing import TypedDict

# Set your API key (Replace with your actual key)
API_KEY = "f668e3202e384bd086093351251003"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

# Function to fetch weather data from WeatherAPI
def get_weather(city: str) -> str:
    url = f"{BASE_URL}?key={API_KEY}&q={city}&aqi=no"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        condition = data["current"]["condition"]["text"]
        temp = data["current"]["temp_c"]
        return f"The weather in {city} is {condition} with a temperature of {temp}°C."
    else:
        return "Unable to fetch weather data."

# Define the state structure
class WeatherState(TypedDict):
    city: str
    response: str

# Create LangGraph state machine
graph = StateGraph(WeatherState)

# Define the function inside the graph
def fetch_weather(state: WeatherState) -> WeatherState:
    state["response"] = get_weather(state["city"])
    return state

# Add function as a node
graph.add_node("fetch_weather", fetch_weather)

# Define the flow
graph.set_entry_point("fetch_weather")
graph.add_edge("fetch_weather", END)

# Compile the graph
weather_graph = graph.compile()

# Run the graph
state = {"city": "Los Angeles", "response": ""}
result = weather_graph.invoke(state)

print(result["response"])


The weather in Los Angeles is Clear with a temperature of 10.6°C.


In [7]:
import requests

# Set your API key (Replace with your actual key)
API_KEY = "f668e3202e384bd086093351251003"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

# Function to check if the city is available
def check_city_weather(city: str):
    url = f"{BASE_URL}?key={API_KEY}&q={city}&aqi=no"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        location = data.get("location", {}).get("name", "Unknown")
        country = data.get("location", {}).get("country", "Unknown")
        temp_c = data.get("current", {}).get("temp_c", "N/A")
        condition = data.get("current", {}).get("condition", {}).get("text", "N/A")
        return f"✅ Weather in {location}, {country}: {condition}, {temp_c}°C"
    else:
        return f"❌ City '{city}' not found in WeatherAPI."

# Check for Salem, India
print(check_city_weather("salem, India"))


✅ Weather in Salem, India: Sunny, 34.7°C


In [9]:
import requests
from langgraph.graph import StateGraph, END
from typing import TypedDict

# Set your API key (Replace with your actual key)
API_KEY = "f668e3202e384bd086093351251003"
BASE_URL = "http://api.weatherapi.com/v1/current.json"

# Function to check if the city exists and fetch weather data
def get_weather(city: str) -> str:
    url = f"{BASE_URL}?key={API_KEY}&q={city}&aqi=no"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if "error" in data:
            return f"❌ City '{city}' not found in WeatherAPI."
        
        location = data["location"]["name"]
        country = data["location"]["country"]
        condition = data["current"]["condition"]["text"]
        temp = data["current"]["temp_c"]
        return f"✅ Weather in {location}, {country}: {condition}, {temp}°C"
    else:
        return "❌ Unable to fetch weather data. Check API key or city name."

# Define the state structure
class WeatherState(TypedDict):
    city: str
    response: str

# Create LangGraph state machine
graph = StateGraph(WeatherState)

# Define the function inside the graph
def fetch_weather(state: WeatherState) -> WeatherState:
    state["response"] = get_weather(state["city"])
    return state

# Add function as a node
graph.add_node("fetch_weather", fetch_weather)

# Define the flow
graph.set_entry_point("fetch_weather")
graph.add_edge("fetch_weather", END)

# Compile the graph
weather_graph = graph.compile()

# Run the graph with a test city
state = {"city": "Los Angeles", "response": ""}
result = weather_graph.invoke(state)

print(result["response"])


✅ Weather in Los Angeles, United States of America: Clear, 10.6°C
